## Basis Swap Par Spreads

The goal of the project is to compute the par spreads of a pool of basis swaps, i.e. those contracts in which two floating libor rates, with different tenors, are exchanged. The par spread is that margin which has to be paid/received on top of the floating rate with the shorter tenor.

The input parameters are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* 3 sets of forward libor rates and their fixing dates, one for the Libor 1M, one for the Libor 3M and one for the Libor 6M.
* The static data of the pool of swaps (nominal, maturity, tenor of the first leg, tenor of the second leg)

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for multi-curve evaluation.

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

In [1]:
from finmarkets import DiscountCurve, OvernightIndexSwap, generate_swap_dates, ForwardRateCurve
from datetime import date

today = date(2013, 10, 31)

ois_quotes = [
    {'maturity': 1, 'rate': 0.00106},
    {'maturity': 2, 'rate': 0.00114},
    {'maturity': 3, 'rate': 0.00115},
    {'maturity': 4, 'rate': 0.00117},
    {'maturity': 5, 'rate': 0.00119},
    {'maturity': 6, 'rate': 0.00121},
    {'maturity': 7, 'rate': 0.00122},
    {'maturity': 8, 'rate': 0.00124},
    {'maturity': 9, 'rate': 0.00128},
    {'maturity': 10, 'rate': 0.00131},
    {'maturity': 11, 'rate': 0.00135},
    {'maturity': 12, 'rate': 0.00138},
    {'maturity': 15, 'rate': 0.00152},
    {'maturity': 18, 'rate': 0.00166},
    {'maturity': 21, 'rate': 0.00184},
    {'maturity': 24, 'rate': 0.00206},
    {'maturity': 36, 'rate': 0.00344},
    {'maturity': 48, 'rate': 0.00543},
    {'maturity': 60, 'rate': 0.00756},
    {'maturity': 72, 'rate': 0.00967},
    {'maturity': 84, 'rate': 0.01162},
    {'maturity': 96, 'rate': 0.0134},
    {'maturity': 108, 'rate': 0.01502},
    {'maturity': 120, 'rate': 0.01649},
    {'maturity': 132, 'rate': 0.01776},
    {'maturity': 144, 'rate': 0.01888},
    {'maturity': 180, 'rate': 0.02137},
    {'maturity': 240, 'rate': 0.02322},
    {'maturity': 300, 'rate': 0.02389},
    {'maturity': 360, 'rate': 0.02416},
]

euribor_1m = [
    {'date': date(2013, 10, 31), 'rate': 0.002},
    {'date': date(2014, 4, 30), 'rate': 0.00431709159584514},
    {'date': date(2014, 10, 30), 'rate': 0.00513437204910293},
    {'date': date(2015, 4, 30), 'rate': 0.00595155807365439},
    {'date': date(2015, 10, 30), 'rate': 0.00676883852691218},
    {'date': date(2016, 4, 30), 'rate': 0.00758611898016997},
    {'date': date(2016, 10, 30), 'rate': 0.00840339943342776},
    {'date': date(2017, 4, 30), 'rate': 0.00922058545797923},
    {'date': date(2017, 10, 30), 'rate': 0.010037865911237},
    {'date': date(2018, 4, 30), 'rate': 0.0108550519357885},
    {'date': date(2018, 10, 30), 'rate': 0.0116723323890463},
    {'date': date(2019, 4, 30), 'rate': 0.0124895184135977},
    {'date': date(2019, 10, 30), 'rate': 0.0133067988668555},
    {'date': date(2020, 4, 30), 'rate': 0.0141240793201133},
    {'date': date(2020, 10, 30), 'rate': 0.0149413597733711},
    {'date': date(2021, 4, 30), 'rate': 0.0157585457979226},
    {'date': date(2021, 10, 30), 'rate': 0.0165758262511804},
    {'date': date(2022, 4, 30), 'rate': 0.0173930122757318},
    {'date': date(2022, 10, 30), 'rate': 0.0182102927289896},
    {'date': date(2023, 4, 30), 'rate': 0.0190274787535411},
    {'date': date(2023, 10, 30), 'rate': 0.0198447592067989},
    {'date': date(2024, 4, 30), 'rate': 0.0206620396600567},
    {'date': date(2024, 10, 30), 'rate': 0.0214793201133144},
    {'date': date(2025, 4, 30), 'rate': 0.0222965061378659},
    {'date': date(2025, 10, 30), 'rate': 0.0231137865911237},
    {'date': date(2026, 4, 30), 'rate': 0.0239309726156752},
    {'date': date(2026, 10, 30), 'rate': 0.024748253068933},
    {'date': date(2027, 4, 30), 'rate': 0.0255654390934844},
    {'date': date(2027, 10, 30), 'rate': 0.0263827195467422},
    {'date': date(2028, 4, 30), 'rate': 0.0267},
]

euribor_3m = [
    {'date': date(2013, 10, 31), 'rate': 0.0035},
    {'date': date(2014, 4, 30), 'rate': 0.00431709159584514},
    {'date': date(2014, 10, 30), 'rate': 0.00513437204910293},
    {'date': date(2015, 4, 30), 'rate': 0.00595155807365439},
    {'date': date(2015, 10, 30), 'rate': 0.00676883852691218},
    {'date': date(2016, 4, 30), 'rate': 0.00758611898016997},
    {'date': date(2016, 10, 30), 'rate': 0.00840339943342776},
    {'date': date(2017, 4, 30), 'rate': 0.00922058545797923},
    {'date': date(2017, 10, 30), 'rate': 0.010037865911237},
    {'date': date(2018, 4, 30), 'rate': 0.0108550519357885},
    {'date': date(2018, 10, 30), 'rate': 0.0116723323890463},
    {'date': date(2019, 4, 30), 'rate': 0.0124895184135977},
    {'date': date(2019, 10, 30), 'rate': 0.0133067988668555},
    {'date': date(2020, 4, 30), 'rate': 0.0141240793201133},
    {'date': date(2020, 10, 30), 'rate': 0.0149413597733711},
    {'date': date(2021, 4, 30), 'rate': 0.0157585457979226},
    {'date': date(2021, 10, 30), 'rate': 0.0165758262511804},
    {'date': date(2022, 4, 30), 'rate': 0.0173930122757318},
    {'date': date(2022, 10, 30), 'rate': 0.0182102927289896},
    {'date': date(2023, 4, 30), 'rate': 0.0190274787535411},
    {'date': date(2023, 10, 30), 'rate': 0.0198447592067989},
    {'date': date(2024, 4, 30), 'rate': 0.0206620396600567},
    {'date': date(2024, 10, 30), 'rate': 0.0214793201133144},
    {'date': date(2025, 4, 30), 'rate': 0.0222965061378659},
    {'date': date(2025, 10, 30), 'rate': 0.0231137865911237},
    {'date': date(2026, 4, 30), 'rate': 0.0239309726156752},
    {'date': date(2026, 10, 30), 'rate': 0.024748253068933},
    {'date': date(2027, 4, 30), 'rate': 0.0255654390934844},
    {'date': date(2027, 10, 30), 'rate': 0.0263827195467422},
    {'date': date(2028, 4, 30), 'rate': 0.0272},
]

euribor_6m = [
    {'date': date(2013, 10, 31), 'rate': 0.005},
    {'date': date(2014, 4, 30), 'rate': 0.0058},
    {'date': date(2014, 10, 30), 'rate': 0.0066},
    {'date': date(2015, 4, 30), 'rate': 0.0074},
    {'date': date(2015, 10, 30), 'rate': 0.0082},
    {'date': date(2016, 4, 30), 'rate': 0.009},
    {'date': date(2016, 10, 30), 'rate': 0.0098},
    {'date': date(2017, 4, 30), 'rate': 0.0106},
    {'date': date(2017, 10, 30), 'rate': 0.0114},
    {'date': date(2018, 4, 30), 'rate': 0.0122},
    {'date': date(2018, 10, 30), 'rate': 0.013},
    {'date': date(2019, 4, 30), 'rate': 0.0138},
    {'date': date(2019, 10, 30), 'rate': 0.0146},
    {'date': date(2020, 4, 30), 'rate': 0.0154},
    {'date': date(2020, 10, 30), 'rate': 0.0162},
    {'date': date(2021, 4, 30), 'rate': 0.017},
    {'date': date(2021, 10, 30), 'rate': 0.0178},
    {'date': date(2022, 4, 30), 'rate': 0.0186},
    {'date': date(2022, 10, 30), 'rate': 0.0194},
    {'date': date(2023, 4, 30), 'rate': 0.0202},
    {'date': date(2023, 10, 30), 'rate': 0.021},
    {'date': date(2024, 4, 30), 'rate': 0.0218},
    {'date': date(2024, 10, 30), 'rate': 0.0226},
    {'date': date(2025, 4, 30), 'rate': 0.0234},
    {'date': date(2025, 10, 30), 'rate': 0.0242},
    {'date': date(2026, 4, 30), 'rate': 0.025},
    {'date': date(2026, 10, 30), 'rate': 0.0258},
    {'date': date(2027, 4, 30), 'rate': 0.0266},
    {'date': date(2027, 10, 30), 'rate': 0.0274},
    {'date': date(2028, 4, 30), 'rate': 0.0282},
]

basis_swaps = [
    {'nominal': 1000000, 'maturity': 12, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 12, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 12, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 24, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 24, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 24, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 60, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 60, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 60, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 90, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 90, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 90, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 120, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 120, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 120, 'first_tenor': 1, 'second_tenor': 6},
]

First of all we need to determine the discount curve from the OIS market quotes using the bootstrap technique.

* generate set of OIS according to the market quotes
* create an objective function with a discount curve and the squared sum of NPVs
* check minimization results
* create the found `DiscountCurve`

In [2]:
from finmarkets import generate_swap_dates, OvernightIndexSwap
pillar_dates = [today]

swaps = []
for quote in ois_quotes:
    swap = OvernightIndexSwap(
           1e6, 
           generate_swap_dates(today, quote['maturity']),
           0.01 * quote['rate'])
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])
    
pillar_dates = sorted(pillar_dates)

In [3]:
from finmarkets import DiscountCurve

def objective_function(x):
    curve = DiscountCurve(today, pillar_dates, x)
    
    sum_sq = 0.0
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq

In [4]:
from scipy.optimize import minimize

x0 = [1.0 for i in range(len(pillar_dates))]
bounds = [(0.01, 100.0) for i in range(len(pillar_dates))]
bounds[0] = (1.0, 1.0)

result = minimize(objective_function, x0, bounds=bounds)

In [5]:
print (result)
print (objective_function(x0))
print (objective_function(result.x))

      fun: 0.0007487920046233709
 hess_inv: <31x31 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.99758165e+05,  2.39068194e-04,  2.40940622e-04,  1.56242586e-04,
        5.21762538e-06,  1.58370994e-04,  6.61528124e-05, -4.61944935e-05,
       -5.31491084e-06,  1.05338915e-05, -8.10116514e-05, -4.08389576e-05,
       -7.27926547e-03,  7.53785749e-04,  6.93722879e-04,  6.81995541e-04,
       -6.58039749e-03, -5.28809586e-03, -3.41212800e-03, -1.16853559e-03,
        5.74333127e-04,  2.06311614e-03,  2.67669737e-03,  2.46575976e-03,
        1.88794498e-03,  3.46496714e-04,  6.23165253e-03,  9.95976366e-03,
        8.77537501e-03, -1.29648649e-03, -1.17902601e-02])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 576
      nit: 6
   status: 0
  success: True
        x: array([1.        , 0.99999911, 0.99999806, 0.99999706, 0.9999961 ,
       0.999995  , 0.99999391, 0.99999281, 0.99999166, 0.99999029,
       0.99998893, 0.99998747, 0.999986  , 0.99998

In [6]:
ois_curve = DiscountCurve(today, pillar_dates, result.x)

In [7]:
from finmarkets import ForwardRateCurve

euribor_curves = {}
pillar_dates = [eur['date'] for eur in euribor_1m]
rates = [eur['rate'] for eur in euribor_1m]
euribor_curves[1] = ForwardRateCurve(pillar_dates, rates)

pillar_dates = [eur['date'] for eur in euribor_3m]
rates = [eur['rate'] for eur in euribor_3m]
euribor_curves[3] = ForwardRateCurve(pillar_dates, rates)

pillar_dates = [eur['date'] for eur in euribor_6m]
rates = [eur['rate'] for eur in euribor_6m]
euribor_curves[6] = ForwardRateCurve(pillar_dates, rates)

Create a BasisSwap class that has a method to compute the fair margin as follows:

$$S = \frac {S_{t2} - S_{t1}}{\text{annuity}_{t1}} $$

In [8]:
bs_mat = [bsw['maturity'] for bsw in basis_swaps]
bs_ft = [bsw['first_tenor'] for bsw in basis_swaps]
bs_st = [bsw['second_tenor'] for bsw in basis_swaps]

In [17]:
from datetime import date
from dateutil.relativedelta import relativedelta
from finmarkets import generate_swap_dates

class BasisRateSwap(object):
    def __init__(self, start_date, notional, 
                 maturity, first_tenor, second_tenor):
    
        self.start_date = today
        self.notional = notional
        self.maturity = maturity
        self.tenor1 = first_tenor
        self.tenor2 = second_tenor
    
        self.tenor1_payment_dates = \
            generate_swap_dates(self.start_date, 
                                self.maturity, 
                                self.tenor1)
        self.tenor2_payment_dates = \
            generate_swap_dates(self.start_date, 
                                self.maturity, 
                                self.tenor2)

    def swap_rate(self, pd, dc, euribor):
        sp = 0 
        for i in range(1, len(pd)):
            F = euribor.forward_rate(pd[i])
            D = dc.df(pd[i])
            tau = (pd[i] - pd[i-1]).days / 360
            sp += F * D * tau
        return sp
        
    def fair_margin(self, discount_curve, euribor_1t, euribor_2t):
    
        delta_sp = self.swap_rate(self.tenor2_payment_dates, discount_curve, euribor_2t) - \
                   self.swap_rate(self.tenor1_payment_dates, discount_curve, euribor_1t) 
        
        annuity_ft = 0
        
        for j in range(1, len(self.tenor1_payment_dates)):
            D = discount_curve.df(self.tenor1_payment_dates[j])
            tau = (self.tenor1_payment_dates[j] - self.tenor1_payment_dates[j-1]).days/360
            annuity_ft += D * tau
        
        return (delta_sp / annuity_ft)

In [18]:
par_spreads = []

for i in range(0, len(bs_mat)):
    par_spreads.append(BasisRateSwap(today, 100, bs_mat[i],
                       bs_ft[i], bs_st[i]).fair_margin(ois_curve, 
                                                       euribor_curves[bs_ft[i]],
                                                       euribor_curves[bs_st[i]]))

print ("Par spreads: " + str(["{:.5f}".format(s) for s in par_spreads]))

Par spreads: ['0.00045', '0.00168', '0.00213', '0.00029', '0.00166', '0.00195', '0.00020', '0.00161', '0.00181', '0.00018', '0.00157', '0.00174', '0.00017', '0.00152', '0.00169']
